In [1]:
import sys, torch, os, time, importlib
import numpy as np
from kilosort import preprocessing, datashift, template_matching, clustering_qr, CCG, clustering_qr, swarmsplitter, spikedetect, run_kilosort


In [2]:
settings = {}
settings['data_folder'] = "/mnt/sorting/kstest/"
settings['probe_path']  = "/mnt/sorting/kstest/neuropixPhase3A_kilosortChanMap.mat"
settings['n_chan_bin'] = 385
settings['fs']       = 30000
settings['nt']     =  61
settings['Th']       = 6
settings['spkTh']    = 10
settings['nwaves']   = 6
settings['nskip']    = 25
settings['nt0min']   = int(20 * settings['nt']/61)
settings['NT']       = 2 * settings['fs']
settings['nblocks']  = 5
settings['binning_depth'] = 5
settings['sig_interp'] = 20


np.random.seed(1)
torch.cuda.manual_seed_all(1)
torch.random.manual_seed(1)
from kilosort import io
importlib.reload(io)
importlib.reload(preprocessing)
importlib.reload(spikedetect)
importlib.reload(template_matching)
importlib.reload(datashift)
importlib.reload(clustering_qr)
importlib.reload(run_kilosort)

probe = io.load_probe(settings['probe_path'])
ops, st, tF, iclust, Wall, is_refractory = run_kilosort.run_kilosort(settings=settings, probe=probe)


In [3]:
from kilosort import io 
importlib.reload(preprocessing)
importlib.reload(io)

root = settings['data_folder']
filename = os.path.join(root, 'smallfile.bin')

probe = io.load_probe(os.path.join(root, 'neuropixPhase3A_kilosortChanMap.mat'))
channel_map = probe['chanMap']
x_chan = probe['xc']
y_chan = probe['yc']

fs = settings['fs']
n_chan_bin = settings['n_chan_bin']
nskip = settings['nskip']

tstart = 5000 
tend = 7000

#device=torch.device('cpu')
device=torch.device('cuda')

In [4]:
# load some example data unfiltered (but mean and median subtracted)
with io.BinaryFiltered(filename, n_chan_bin=n_chan_bin, fs=fs, channel_map=channel_map, device=device) as bfile:
    data_raw = bfile[tstart : tend].cpu().numpy()

/home/shashwat/repositories/kilosort_dev/kilosort/io.py:351: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  X = torch.from_numpy(samples.T).to(self.device).float()


In [5]:
# compute filters (only needs to happen once and be saved)
hp_filter = preprocessing.get_highpass_filter(fs, device=device)
with io.BinaryFiltered(filename, n_chan_bin=n_chan_bin, fs=fs, channel_map=channel_map,
                        hp_filter=hp_filter, device=device) as bfile:
    whiten_mat = preprocessing.get_whitening_matrix(bfile, x_chan, y_chan, 
                                                        nskip = nskip)

In [6]:
# load example data and filter
with io.BinaryFiltered(filename, n_chan_bin=n_chan_bin,  fs=fs, channel_map=channel_map, 
                        hp_filter=hp_filter, whiten_mat=whiten_mat, device=device) as bfile:
    data_filt = bfile[tstart : tend].cpu().numpy()

In [7]:
to_display = [i for i in range(data_raw.shape[0])]
print(all([isinstance(i, int) for i in to_display]))
data_filt[to_display, 10:20].shape

True


(384, 10)

In [ ]:
print(type(data_filt))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
plt.imshow(data_raw, aspect='auto', vmin=-40, vmax=40, cmap='bwr')
plt.subplot(2,1,2)
plt.imshow(data_filt, aspect='auto', vmin=-5, vmax=5, cmap='bwr')

In [ ]:
sorting /media/carsen/ssd3/spikesorting/kstest/Hopkins_probe00_lil.bin
0.5216231346130371
28.16948699951172
95 95 0.0 9088 1.0
182 93 6410.0 6961 0.9382675886154175
259 95 11270.0 4974 0.9135704040527344
325 93 10781.0 3048 0.9121253490447998
362 95 15942.0 3667 0.892087459564209
396 94 13479.0 2314 0.8960741758346558
411 94 13269.0 2389 0.9001560807228088
407 94 16949.0 3243 0.8861321210861206
407 94 14217.0 2201 0.8897771835327148
415 95 15811.0 2395 0.8831503391265869
423 95 18406.0 3325 0.8790006637573242
420 95 15438.0 2196 0.8871212005615234
422 93 16065.0 2053 0.8814165592193604
422 95 13556.0 1904 0.8914451003074646
425 95 15562.0 2030 0.8840786814689636
406 93 15166.0 1995 0.8875391483306885
397 94 12212.0 1614 0.885621190071106
397 92 16179.0 2032 0.8826791048049927
392 93 15743.0 1994 0.8847112655639648
404 94 15868.0 1842 0.8832195401191711
410 93 14277.0 1810 0.8859168291091919
409 94 17708.0 2416 0.8737927675247192
416 93 15803.0 1985 0.8827481269836426
410 94 14156.0 1751 0.8895689249038696
403 93 17161.0 2252 0.8749827742576599
411 95 14394.0 1655 0.8873156309127808
395 94 14424.0 1705 0.8867697715759277
410 94 14369.0 1761 0.8846805095672607
413 93 16110.0 2068 0.8772283792495728
403 94 14792.0 1796 0.885819673538208
407 95 16168.0 2105 0.8819477558135986
315 15997.0 1836 0.879417359828949
266 16288.0 2145 0.87995445728302
246 14555.0 1950 0.8845446109771729
238 18433.0 2618 0.8702524900436401
237 15227.0 2048 0.8811249732971191
236 15984.0 2059 0.8784278631210327
235 14320.0 1802 0.8859401941299438
234 15123.0 1821 0.8824591636657715
234 15843.0 1973 0.8772786259651184
0
0 6 0.34157443046569824
10 42 3.874079942703247
20 72 6.716186285018921
40 158 12.755425691604614
50 165 14.007964372634888
60 211 16.349457263946533
80 268 20.716758966445923
90 357 24.657107830047607
0 1.0
100 69.0
200 135.0
300 199.0
380 247.0